In [2]:
import numpy as np 
import tensorflow as tf # use the tf2.4
import tensorflow_datasets as tfds 
import tensorflow_addons as tfa
from adabelief_tf import AdaBeliefOptimizer
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score
from random import shuffle

from torch.utils.tensorboard import SummaryWriter
from collections import Counter

2022-07-28 15:17:59.137296: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Hier werden wir das Maml — Modell so trainieren, dass wir eine Multiclass — Accuracy bekommen. Dazu verwenden wir One vs Rest und One vs One Methoden wie in https://scikit-learn.org/stable/modules/multiclass.html dokumentiert.

### Daten laden

In [3]:
!nvidia-smi


Thu Jul 28 15:18:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   30C    P8    14W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   

In [4]:
(kmnist_tr, kmnist_ts) = tfds.load('kmnist', 
                                   split=['train','test'],
                                   shuffle_files=True,
                                   as_supervised=True,
                                   with_info=False,)

kmnist_tr = kmnist_tr.apply(lambda ds: ds.map(lambda x, y: (x, y + 60)))
kmnist_ts = kmnist_ts.apply(lambda ds: ds.map(lambda x, y: (x, y + 60)))



(emnist_tr, emnist_ts) = tfds.load('emnist', 
                                   split=['train','test'],
                                   shuffle_files=True,
                                   as_supervised=True,
                                   with_info=False,)

kmnist_tr = kmnist_tr.concatenate(emnist_tr).shuffle(100000)
kmnist_ts = kmnist_ts.concatenate(emnist_ts).shuffle(100000)


kmnist_tr = kmnist_tr.apply(lambda ds: ds.map(
        lambda x, y: 
            (tf.reshape(x, [1, 28, 28, 1]), y)
        ))
kmnist_ts = kmnist_ts.apply(lambda ds: ds.map(
        lambda x, y: 
            (tf.reshape(x, [1, 28, 28, 1]), y)
        ))

out_of_distro_classes_list = [60, 61, 5, 14, 17]
out_of_distro_tr = dict()
in_of_distro_tr = dict()

#collects all classes for out_of_distro_tr
# Todo: (5, 14, 17 funktioniert nicht)
for ofd in out_of_distro_classes_list:
    out_of_distro_tr[ofd] = kmnist_tr.filter(lambda x, y: int(y) == ofd)
    out_of_distro_tr[ofd] = out_of_distro_tr[ofd].shuffle(1000)

    
# collects all opposite classes for in_of_distro_tr
for ofd in out_of_distro_classes_list:
    is_first_time = True
    for ofd_inner in out_of_distro_classes_list:
        if ofd_inner == ofd:
            continue
        if is_first_time:
            in_of_distro_tr[ofd] = out_of_distro_tr[ofd_inner]
            is_first_time = False
        else:
            in_of_distro_tr[ofd] = in_of_distro_tr[ofd].concatenate(out_of_distro_tr[ofd_inner])
    in_of_distro_tr[ofd] = in_of_distro_tr[ofd].shuffle(1000)


2022-07-28 15:18:23.988289: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-28 15:18:31.076384: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12848 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:3b:00.0, compute capability: 7.5
2022-07-28 15:18:31.078650: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 12848 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:af:00.0, compute capability: 7.5
2022-07-28 15:18:31.080692: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU

### Äußeres Modell trainieren

In [6]:

inner_lr = 1E-5
outter_lr = 1E-5
inner_task_loop_no = 64 # give 32 tasks of 2ways-1shot
# opt_inner = tf.keras.optimizers.Adagrad(inner_lr)
# opt_outter = tf.keras.optimizers.Adagrad(outter_lr, clipnorm=1.)
opt_inner = tfa.optimizers.NovoGrad(inner_lr)
opt_outter = tfa.optimizers.NovoGrad(outter_lr, clipnorm=1.)
# opt_inner = tfa.optimizers.RectifiedAdam(inner_lr)
# opt_outter = tfa.optimizers.RectifiedAdam(outter_lr, clipnorm=1.)
# opt_inner = AdaBeliefOptimizer(inner_lr)
# opt_outter = AdaBeliefOptimizer(outter_lr)
# opt_inner = tf.keras.optimizers.SGD(inner_lr, momentum=.0)
# opt_outter = tf.keras.optimizers.SGD(outter_lr, momentum=.9, clipnorm=1.)

#opt_outter = tfa.optimizers.MovingAverage(opt_outter) # using average strategy in tfa


def cnn():
    Input = tf.keras.Input([28, 28, 1])
    Input_n = Input/128.0 - 1
    conv1 = tf.keras.layers.Conv2D(32, [3, 3], strides=(2, 2), padding="SAME", kernel_initializer="he_uniform", activation=tf.nn.relu)(Input_n) #[14,14]
    conv2 = tf.keras.layers.Conv2D(64, [3, 3], strides=(2, 2), padding="SAME", kernel_initializer="he_uniform", activation=tf.nn.relu)(conv1) #[7,7]
    conv3 = tf.keras.layers.Conv2D(128, [3, 3], strides=(2, 2), padding="SAME", kernel_initializer="he_uniform", activation=tf.nn.relu)(conv2) #[4,4]
    fc = tf.keras.layers.Flatten()(conv3)
    fc1 = tf.keras.layers.Dense(128, kernel_initializer="he_uniform", activation=tf.nn.relu)(fc)
    fc2 = tf.keras.layers.Dense(256, kernel_initializer="he_uniform", activation=tf.nn.relu)(fc1)
    fc3 = tf.keras.layers.Dense(512, kernel_initializer="he_uniform", activation=tf.nn.relu)(fc2)
    out = tf.keras.layers.Dense(1, kernel_initializer="he_uniform", activation=None)(fc3)
    return tf.keras.Model(inputs=Input, outputs=out)
 
def select_support_query_set(dataset_iter):
    flag_1 = flag_0 = True
    imgs_1_support, labs_1_support = next(dataset_iter)
    imgs_0_support, labs_0_support = next(dataset_iter)
    support_set = tf.concat([imgs_1_support, imgs_0_support], axis=0)
    while(flag_1 or flag_0):
        imgs_query, labs_query = next(dataset_iter)
        if ((labs_query.numpy() == labs_1_support.numpy()) and flag_1) :
            imgs_1_query = tf.Variable(imgs_query)
            flag_1 = False
        elif ((labs_query.numpy() == labs_0_support.numpy()) and flag_0) :
            imgs_0_query = tf.Variable(imgs_query)
            flag_0 = False
        pass 
    pass 
    query_set = tf.concat([imgs_1_query, imgs_0_query], axis=0)
    return [support_set, query_set]
 
# Task hint:
# this example will use 2ways-1shot for training the MAML.
# the inner task loop will set as 30. 
kmnist_tr = kmnist_tr.batch(1).repeat()
kmnist_tr_iter = iter(kmnist_tr)
meta_labs = tf.Variable([[1], [0]], dtype=tf.float32)

cnn_model = cnn()

# according to the TF2 manual, the variable of the model would need to be
# initialized, and use it to inferece something is a way to initialize 
# the varialbe.
support_set, query_set = select_support_query_set(kmnist_tr_iter)
_ = cnn_model(support_set) # 1)for initializing the model. 2)for define the loss function out-of the loop 
loss_inner_meta = lambda: tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=meta_labs, logits=cnn_model(support_set)))
loss_inner_task = lambda: tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=meta_labs, logits=cnn_model(query_set)))



2022-07-26 14:38:38.846207: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201


In [4]:

for step in range(50000):
    # keep the original weights for meta-training
    meta_weights = [tf.Variable(target_weights) for target_weights in cnn_model.trainable_weights]
    
    def meta_loss():
        loss_outter = 0
        for task_count in range(inner_task_loop_no):
            # fetch the dataset. The first dataset is always be 1. 
            # So, the labels will be defined as the meta-labels insteadly. 
            support_set, query_set = select_support_query_set(kmnist_tr_iter)
            
            # put the meta-weights into the model
            for model_weight_index in range(len(cnn_model.trainable_weights)):
                cnn_model.trainable_weights[model_weight_index].assign(meta_weights[model_weight_index])
            pass

            # updateing the weights from meta-weights
            opt_inner.minimize(loss=loss_inner_meta, var_list=cnn_model.trainable_weights) 
            
            # calculating the task loss (after the meta-weights updating)
            # here, the same support set and query set are used. you can 
            # also creat a different query set for meta-weight training.
            loss_outter += loss_inner_task()
        pass 
        return loss_outter/inner_task_loop_no
    pass 

    # put the meta-weights into the model
    for model_weight_index in range(len(cnn_model.trainable_weights)):
        cnn_model.trainable_weights[model_weight_index].assign(meta_weights[model_weight_index])
    pass

    # update the meta-weights
    opt_outter.minimize(loss=meta_loss, var_list=cnn_model.trainable_weights) 
    
    if step > 2:
        break

    print("step: {}  outter_loss: {}".format(step, meta_loss()))
   #summary_writer.add_scalar("t

step: 0  outter_loss: 0.661101222038269
step: 1  outter_loss: 0.6601587533950806
step: 2  outter_loss: 0.6592254638671875


## OneVs Rest

Änderung:
- kmnist_tr und kmnist_ts wurden die Form der x auf [1, 28, 28 ,1] geändert
- Reihenfolge der Datasets beachten! Erst nach Konkenation in und out_distro bilden

### One Vs Rest wirklich

In [ ]:
N_TRAINING_ONE_IN = 3
N_TRAINING_ONE_REST = 10

In [10]:
cloned_models = dict()
for ofd in out_of_distro_classes_list:
    cloned_models[ofd] = tf.keras.models.clone_model(cnn_model)
    cloned_models[ofd].compile( tf.keras.optimizers.SGD(), tf.keras.losses.BinaryCrossentropy(from_logits=True))

    training_elements = list()
    
    ## todo: warum 8, dann vier. _Vermutlich out_distribution
    for i, (x, _) in enumerate(out_of_distro_tr[ofd]):
        if i >= N_TRAINING_ONE_IN:
            break
        training_elements.append((x, tf.ones([1,1])))    for i, (x, _) in enumerate(in_of_distro_tr[ofd]):
        if i >= N_TRAINING_ONE_REST:
            break
        training_elements.append((x, tf.zeros([1, 1])))

    shuffle(training_elements)
    print("ofd", ofd)
    for x, y in training_elements:
        cloned_models[ofd].fit(x, y)

60 \//// 6
1/1 [==============================] - 0s 7ms/step - loss: 0.3311
61 \//// 6
1/1 [==============================] - 0s 7ms/step - loss: 0.4217
5 \//// 6
1/1 [==============================] - 0s 7ms/step - loss: 0.9786
14 \//// 6
1/1 [==============================] - 0s 7ms/step - loss: 3.4289


2022-07-26 14:42:49.480394: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 779 of 1000
2022-07-26 14:42:51.543428: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


17 \//// 6
1/1 [==============================] - 0s 7ms/step - loss: 0.4429


In [34]:
for _, y in training_elements:
    print(y)

tf.Tensor([[0.]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.]], shape=(1, 1), dtype=float32)


In [11]:
for x, _ in kmnist_ts:
    cloned_models[ofd](x)
    break

In [18]:
list_pred_classes = list()
list_true_classes = list()
for i, (x, y) in enumerate(kmnist_ts):
    if i > 100:
        break
    predictions = list()
    for ofd in out_of_distro_classes_list:
        pred_ofd = cloned_models[ofd](x)
        predictions.append((ofd, pred_ofd))

    max_pred_class, _ = max(predictions, key=lambda x: x[1])
    list_true_classes.append(int(y))
    list_pred_classes.append(max_pred_class)

accuracy = accuracy_score(list_true_classes, list_pred_classes)



In [19]:
accuracy, list_true_classes, list_pred_classes

(0.009900990099009901,
 [55,
  8,
  68,
  3,
  65,
  21,
  67,
  12,
  55,
  5,
  4,
  6,
  28,
  58,
  0,
  8,
  8,
  55,
  61,
  1,
  62,
  4,
  30,
  13,
  14,
  53,
  28,
  43,
  0,
  8,
  7,
  2,
  18,
  21,
  27,
  9,
  3,
  33,
  6,
  4,
  40,
  43,
  18,
  8,
  13,
  62,
  11,
  28,
  0,
  63,
  63,
  18,
  6,
  7,
  8,
  36,
  4,
  60,
  11,
  65,
  6,
  5,
  38,
  2,
  36,
  2,
  12,
  40,
  63,
  9,
  24,
  10,
  7,
  52,
  4,
  67,
  5,
  18,
  41,
  23,
  2,
  5,
  4,
  69,
  3,
  67,
  43,
  0,
  40,
  32,
  16,
  43,
  30,
  25,
  62,
  38,
  39,
  40,
  24,
  24,
  22],
 [61,
  17,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  17,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  17,
  61,
  61,
  17,
  61,
  61,
  17,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  17,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,
  61,

## OnevsOne

In [21]:
used_pairs = list()
cloned_models = dict()

for ofd_outer in out_of_distro_classes_list:
    for ofd_inner in out_of_distro_classes_list:
        print(ofd_outer, ofd_innerb)
        if (ofd_outer, ofd_inner) in used_pairs or \
            (ofd_inner, ofd_outer) in used_pairs or \
            ofd_inner == ofd_outer:
            break

        used_pairs.append((ofd_outer, ofd_inner))
        training_elements = list()

        for i, (x, _) in enumerate(out_of_distro_tr[ofd_outer]):
            if i >= 3:
                break
            training_elements.append((x, tf.ones([1,1])))

        for i, (x, _) in enumerate(in_of_distro_tr[ofd_inner]):
            if i >= 3:
                break
            training_elements.append((x, tf.zeros([1, 1])))

        cloned_models[(ofd_outer, ofd_inner)] = tf.keras.models.clone_model(cnn_model)
        cloned_models[(ofd_outer, ofd_inner)].compile( 
            tf.keras.optimizers.SGD(), 
            tf.keras.losses.BinaryCrossentropy(from_logits=True)
            )

        shuffle(training_elements)

        for x, y in training_elements:
            cloned_models[(ofd_outer, ofd_inner)].fit(x, y)


1/1 [==============================] - 0s 7ms/step - loss: 0.4826


2022-07-26 15:20:03.695317: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 987 of 1000
2022-07-26 15:20:03.841534: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


1/1 [==============================] - 0s 7ms/step - loss: 1.2773


2022-07-26 15:20:18.333604: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 970 of 1000
2022-07-26 15:20:18.552304: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


1/1 [==============================] - 0s 8ms/step - loss: 0.8629


...

1/1 [==============================] - 0s 6ms/step - loss: 0.1720


2022-07-26 15:21:21.610227: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 686 of 1000
2022-07-26 15:21:24.830812: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


1/1 [==============================] - 0s 7ms/step - loss: 0.3227


2022-07-26 15:21:40.168092: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 585 of 1000
2022-07-26 15:21:44.504907: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


1/1 [==============================] - 0s 6ms/step - loss: 0.8739


In [28]:
used_pairs

[(61, 60),
 (5, 60),
 (5, 61),
 (14, 60),
 (14, 61),
 (14, 5),
 (17, 60),
 (17, 61),
 (17, 5),
 (17, 14)]

In [26]:
list_pred_classes = list()
list_true_classes = list()

for i, (x, y) in enumerate(kmnist_ts):
    if i > 100:
        break

    predictions_votes = list()

    for ofd_outter, ofd_inner in cloned_models:
        pred = cloned_models[(ofd_outter, ofd_inner)](x)
        if pred >= 0.5:
            predictions_votes.append(ofd_outter)
        else:
            predictions_votes.append(ofd_inner)

    ##calculates, which vote / elemenst is most frequent
    prediction_counter = Counter(predictions_votes)
    most_common_vote = prediction_counter.most_common(1)[0][0]

    list_pred_classes.append(most_common_vote)
    list_true_classes.append(int(y))

accuracy = accuracy_score(list_true_classes, list_pred_classes)


In [27]:
accuracy, list_true_classes, list_pred_classes

(0.009900990099009901,
 [30,
  5,
  4,
  20,
  14,
  7,
  0,
  3,
  37,
  66,
  39,
  5,
  4,
  2,
  53,
  32,
  15,
  1,
  55,
  39,
  61,
  55,
  55,
  2,
  9,
  30,
  7,
  43,
  7,
  47,
  3,
  0,
  7,
  10,
  7,
  3,
  49,
  8,
  6,
  6,
  2,
  7,
  61,
  49,
  9,
  49,
  64,
  15,
  63,
  40,
  2,
  13,
  0,
  6,
  6,
  18,
  11,
  2,
  34,
  64,
  53,
  12,
  8,
  40,
  63,
  29,
  0,
  5,
  50,
  65,
  9,
  62,
  4,
  6,
  3,
  48,
  65,
  0,
  29,
  40,
  7,
  18,
  12,
  40,
  33,
  1,
  64,
  1,
  3,
  66,
  0,
  24,
  7,
  40,
  25,
  4,
  42,
  2,
  2,
  0,
  42],
 [61,
  60,
  5,
  60,
  5,
  61,
  60,
  60,
  5,
  61,
  61,
  60,
  60,
  60,
  60,
  61,
  14,
  5,
  61,
  60,
  61,
  5,
  60,
  60,
  61,
  60,
  60,
  60,
  60,
  61,
  5,
  60,
  60,
  60,
  14,
  60,
  60,
  61,
  5,
  60,
  60,
  61,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  61,
  61,
  5,
  5,
  60,
  60,
  60,
  60,
  61,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  14,
